# Shift Data to Historical Records

In [1]:
import pathlib
import sys

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.mlab as mlab

%matplotlib inline

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
#add the 'src' directory to path to import modules
PROJECT_DIR = pathlib.Path.cwd().resolve().parent
sys.path.append(str(PROJECT_DIR))

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

DATA_DIR = PROJECT_DIR / 'data'
SCOPED_DATA_DIR = DATA_DIR / '03-scoped'
PROCESSED_DATA_DIR = DATA_DIR / '04-processed'

In [2]:
league = 'english-premier-league'
load_fp = SCOPED_DATA_DIR / league / str(league + '-scoped-1.csv')
seasons = ['2009-2010', '2010-2011', '2011-2012']


In [3]:
df_orig = pd.read_csv(load_fp, parse_dates = ['date'], index_col=None)
df_orig = df_orig[df_orig['season'].isin(seasons)]
# Calculate Results column
conditions = [df_orig['h_ftgoals'] > df_orig['a_ftgoals'],
              df_orig['h_ftgoals'] == df_orig['a_ftgoals'],
              df_orig['h_ftgoals'] < df_orig['a_ftgoals']]
choices = ['hwin', 'draw', 'awin']
df_orig['result'] = np.select(conditions, choices, default='not-played')
df_orig.head()

,season,date,h,a,h_ftgoals,a_ftgoals,h_htgoals,a_htgoals,h_corners,h_fouls,h_ycards,h_rcards,h_shots,h_sot,a_corners,a_fouls,a_ycards,a_rcards,a_shots,a_sot,odds_hwin_bbmean,odds_draw_bbmean,odds_awin_bbmean,odds_hwin_bbmax,odds_draw_bbmax,odds_awin_bbmax,odds_hwin_WH,odds_draw_WH,odds_awin_WH,odds_ftgoalsu2.5_bbmax,odds_ftgoalso2.5_bbmax,odds_ftgoalsu2.5_bbmean,odds_ftgoalso2.5_bbmean,heatmap_path,shotmap_path,result
0,2009-2010,2009-08-15,aston-villa,wigan-athletic,0.0,2.0,0.0,1.0,4.0,15.0,2.0,0.0,11.0,5.0,6.0,14.0,2.0,0.0,14.0,7.0,1.66,3.51,5.33,1.72,3.77,6.05,1.70,3.4,5.50,1.91,2.04,1.81,1.92,/data/02-interim/football-data/english-premier...,/data/02-interim/football-data/english-premier...,awin
1,2009-2010,2009-08-15,blackburn-rovers,manchester-city,0.0,2.0,0.0,1.0,5.0,12.0,2.0,0.0,17.0,9.0,4.0,9.0,1.0,0.0,8.0,5.0,3.37,3.24,2.12,3.78,3.35,2.25,3.50,3.2,2.15,1.78,2.15,1.71,2.04,/data/02-interim/football-data/english-premier...,/data/02-interim/football-data/english-premier...,awin
2,2009-2010,2009-08-15,bolton-wanderers,sunderland,0.0,1.0,0.0,1.0,4.0,16.0,2.0,0.0,11.0,3.0,7.0,10.0,1.0,0.0,20.0,13.0,2.24,3.20,3.15,2.37,3.31,3.40,2.30,3.2,3.20,1.70,2.25,1.66,2.12,/data/02-interim/football-data/english-premier...,/data/02-interim/football-data/english-premier...,awin
3,2009-2010,2009-08-15,chelsea,hull-city,2.0,1.0,1.0,1.0,12.0,13.0,1.0,0.0,26.0,12.0,4.0,15.0,2.0,0.0,7.0,3.0,1.17,6.26,16.39,1.20,7.71,23.14,1.17,6.5,21.00,2.44,1.67,2.26,1.58,/data/02-interim/football-data/english-premier...,/data/02-interim/football-data/english-premier...,hwin
4,2009-2010,2009-08-15,everton,arsenal,1.0,6.0,0.0,3.0,4.0,11.0,0.0,0.0,8.0,5.0,9.0,13.0,0.0,0.0,15.0,9.0,3.07,3.21,2.28,3.34,3.34,2.42,3.20,3.2,2.30,1.77,2.20,1.70,2.06,/data/02-interim/football-data/english-premier...,/data/02-interim/football-data/english-premier...,awin


In [29]:
season_df = df_orig.copy(deep=True)
# 2012-2013 Season ios the first season where we have got the Pinnacle Closing odds
get_season = ['2009-2010']
# # get_season = ['2012-2013', '2013-2014', '2014-2015']
season_df = season_df[season_df['season'].isin(get_season)].sort_values(by=['date'])
keeper_cols = ['h', 'a', 'date', 'h_ftgoals', 'a_ftgoals', 'h_shots', 'a_shots',
               'odds_hwin_bbmean', 'odds_draw_bbmean', 'odds_awin_bbmean' ]
season_df = season_df[keeper_cols]
conditions = [season_df['h_ftgoals'] > season_df['a_ftgoals'],
             season_df['h_ftgoals'] == season_df['a_ftgoals'],
             season_df['h_ftgoals'] < season_df['a_ftgoals']]
choices = ['hwin', 'draw', 'awin']
season_df['result'] = np.select(conditions, choices)
season_df.rename(columns={'date_fdcu': 'date',
                          'odds_hwin_bbmean': 'odds_hwin',
                          'odds_draw_bbmean': 'odds_draw',
                          'odds_awin_bbmean': 'odds_awin'}, inplace=True)
season_df.sort_values(by='date', inplace=True)
season_df.reset_index(drop=True, inplace=True)
season_df.head(50)

,h,a,date,h_ftgoals,a_ftgoals,h_shots,a_shots,odds_hwin,odds_draw,odds_awin,result
0,aston-villa,wigan-athletic,2009-08-15,0.0,2.0,11.0,14.0,1.66,3.51,5.33,awin
1,blackburn-rovers,manchester-city,2009-08-15,0.0,2.0,17.0,8.0,3.37,3.24,2.12,awin
2,bolton-wanderers,sunderland,2009-08-15,0.0,1.0,11.0,20.0,2.24,3.20,3.15,awin
3,chelsea,hull-city,2009-08-15,2.0,1.0,26.0,7.0,1.17,6.26,16.39,hwin
4,everton,arsenal,2009-08-15,1.0,6.0,8.0,15.0,3.07,3.21,2.28,awin
5,portsmouth,fulham,2009-08-15,0.0,1.0,16.0,9.0,2.53,3.18,2.73,awin
6,stoke-city,burnley,2009-08-15,2.0,0.0,12.0,9.0,1.91,3.29,4.02,hwin
7,wolverhampton-wanderers,west-ham-united,2009-08-15,0.0,2.0,19.0,16.0,2.57,3.19,2.68,awin
8,manchester-united,birmingham-city,2009-08-16,1.0,0.0,26.0,6.0,1.19,5.91,14.99,hwin
9,tottenham-hotspur,liverpool,2009-08-16,2.0,1.0,17.0,6.0,3.13,3.22,2.23,hwin


In [5]:
season_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 11 columns):
h            380 non-null object
a            380 non-null object
date         380 non-null datetime64[ns]
h_ftgoals    380 non-null float64
a_ftgoals    380 non-null float64
h_shots      380 non-null float64
a_shots      380 non-null float64
odds_hwin    380 non-null float64
odds_draw    380 non-null float64
odds_awin    380 non-null float64
result       380 non-null object
dtypes: datetime64[ns](1), float64(7), object(3)
memory usage: 32.7+ KB


### Pare down Data

### Shape Season to historical data

In [6]:
def get_historical_records(group, home_or_away):
    features = [col for col in group.columns if '_' in col]
    for feat in features:
        new_feat_cols = [feat + '_' + str(n) for n, feat in zip(range (0,-len(group), -1), [feat]*len(group))]
        for shift_n, new_feat_col in enumerate(new_feat_cols):
            group[new_feat_col] = group[feat]
            group[new_feat_col] = group[new_feat_col].shift(shift_n)
        group.drop(columns=[feat], inplace=True)
    return group


def get_records(df, home_or_away, loc_record):
    feature_cols = [col for col in df.columns if loc_record in col]
    feature_cols.insert(0, home_or_away)
    cut_df = df[feature_cols]
    records = cut_df.groupby(by=home_or_away, sort=True).apply(get_historical_records, home_or_away)
    return records
    
def form_historical_records(df):
    h_teams = df['h'].values
    a_teams = df['a'].values
    # Add
    dates = df['date'].values
    hwin_odds = df['odds_hwin'].values ; draw_odds = df['odds_draw'].values ; awin_odds = df['odds_awin'].values
    result = df['result'].values
    record_dfs = []
    features = []
    for home_or_away in ['h', 'a']:
        for loc_record in ['h_', 'a_']:
            record_df = get_records(df, home_or_away, loc_record)
            new_cols = {col: home_or_away + '_' + col for col in record_df.columns if col not in home_or_away}
            record_df.rename(columns=new_cols, inplace=True)
            record_dfs.append(record_df)
            features.extend([col for col in record_df.columns if loc_record in col])

    full_records = pd.concat(record_dfs, axis=1, sort=True)
    # Drop any duplicate columns - h and a get duplicated as a multiple of number of features
    full_records = full_records.loc[:,~full_records.columns.duplicated()]
    full_records['date'] = dates
    full_records['odds_hwin'] = hwin_odds
    full_records['odds_draw'] = draw_odds
    full_records['odds_awin'] = awin_odds
    full_records['result'] = result

    return full_records

In [7]:
df = season_df.copy(deep=True)
df = form_historical_records(df)
df.head()

,h,h_h_ftgoals_0,h_h_ftgoals_-1,h_h_ftgoals_-2,h_h_ftgoals_-3,h_h_ftgoals_-4,h_h_ftgoals_-5,h_h_ftgoals_-6,h_h_ftgoals_-7,h_h_ftgoals_-8,h_h_ftgoals_-9,h_h_ftgoals_-10,h_h_ftgoals_-11,h_h_ftgoals_-12,h_h_ftgoals_-13,h_h_ftgoals_-14,h_h_ftgoals_-15,h_h_ftgoals_-16,h_h_ftgoals_-17,h_h_ftgoals_-18,h_h_shots_0,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_a_ftgoals_0,h_a_ftgoals_-1,h_a_ftgoals_-2,h_a_ftgoals_-3,h_a_ftgoals_-4,h_a_ftgoals_-5,h_a_ftgoals_-6,h_a_ftgoals_-7,h_a_ftgoals_-8,h_a_ftgoals_-9,h_a_ftgoals_-10,h_a_ftgoals_-11,h_a_ftgoals_-12,h_a_ftgoals_-13,h_a_ftgoals_-14,h_a_ftgoals_-15,h_a_ftgoals_-16,h_a_ftgoals_-17,h_a_ftgoals_-18,h_a_shots_0,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_shots_-4,h_a_shots_-5,h_a_shots_-6,h_a_shots_-7,h_a_shots_-8,h_a_shots_-9,h_a_shots_-10,h_a_shots_-11,h_a_shots_-12,h_a_shots_-13,h_a_shots_-14,h_a_shots_-15,h_a_shots_-16,h_a_shots_-17,h_a_shots_-18,a,a_h_ftgoals_0,a_h_ftgoals_-1,a_h_ftgoals_-2,a_h_ftgoals_-3,a_h_ftgoals_-4,a_h_ftgoals_-5,a_h_ftgoals_-6,a_h_ftgoals_-7,a_h_ftgoals_-8,a_h_ftgoals_-9,a_h_ftgoals_-10,a_h_ftgoals_-11,a_h_ftgoals_-12,a_h_ftgoals_-13,a_h_ftgoals_-14,a_h_ftgoals_-15,a_h_ftgoals_-16,a_h_ftgoals_-17,a_h_ftgoals_-18,a_h_shots_0,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_shots_-4,a_h_shots_-5,a_h_shots_-6,a_h_shots_-7,a_h_shots_-8,a_h_shots_-9,a_h_shots_-10,a_h_shots_-11,a_h_shots_-12,a_h_shots_-13,a_h_shots_-14,a_h_shots_-15,a_h_shots_-16,a_h_shots_-17,a_h_shots_-18,a_a_ftgoals_0,a_a_ftgoals_-1,a_a_ftgoals_-2,a_a_ftgoals_-3,a_a_ftgoals_-4,a_a_ftgoals_-5,a_a_ftgoals_-6,a_a_ftgoals_-7,a_a_ftgoals_-8,a_a_ftgoals_-9,a_a_ftgoals_-10,a_a_ftgoals_-11,a_a_ftgoals_-12,a_a_ftgoals_-13,a_a_ftgoals_-14,a_a_ftgoals_-15,a_a_ftgoals_-16,a_a_ftgoals_-17,a_a_ftgoals_-18,a_a_shots_0,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shots_-17,a_a_shots_-18,date,odds_hwin,odds_draw,odds_awin,result
0,aston-villa,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wigan-athletic,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,1.66,3.51,5.33,awin
1,blackburn-rovers,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manchester-city,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,3.37,3.24,2.12,awin
2,bolton-wanderers,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sunderland,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1

In [8]:
df.tail()

,h,h_h_ftgoals_0,h_h_ftgoals_-1,h_h_ftgoals_-2,h_h_ftgoals_-3,h_h_ftgoals_-4,h_h_ftgoals_-5,h_h_ftgoals_-6,h_h_ftgoals_-7,h_h_ftgoals_-8,h_h_ftgoals_-9,h_h_ftgoals_-10,h_h_ftgoals_-11,h_h_ftgoals_-12,h_h_ftgoals_-13,h_h_ftgoals_-14,h_h_ftgoals_-15,h_h_ftgoals_-16,h_h_ftgoals_-17,h_h_ftgoals_-18,h_h_shots_0,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_a_ftgoals_0,h_a_ftgoals_-1,h_a_ftgoals_-2,h_a_ftgoals_-3,h_a_ftgoals_-4,h_a_ftgoals_-5,h_a_ftgoals_-6,h_a_ftgoals_-7,h_a_ftgoals_-8,h_a_ftgoals_-9,h_a_ftgoals_-10,h_a_ftgoals_-11,h_a_ftgoals_-12,h_a_ftgoals_-13,h_a_ftgoals_-14,h_a_ftgoals_-15,h_a_ftgoals_-16,h_a_ftgoals_-17,h_a_ftgoals_-18,h_a_shots_0,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_shots_-4,h_a_shots_-5,h_a_shots_-6,h_a_shots_-7,h_a_shots_-8,h_a_shots_-9,h_a_shots_-10,h_a_shots_-11,h_a_shots_-12,h_a_shots_-13,h_a_shots_-14,h_a_shots_-15,h_a_shots_-16,h_a_shots_-17,h_a_shots_-18,a,a_h_ftgoals_0,a_h_ftgoals_-1,a_h_ftgoals_-2,a_h_ftgoals_-3,a_h_ftgoals_-4,a_h_ftgoals_-5,a_h_ftgoals_-6,a_h_ftgoals_-7,a_h_ftgoals_-8,a_h_ftgoals_-9,a_h_ftgoals_-10,a_h_ftgoals_-11,a_h_ftgoals_-12,a_h_ftgoals_-13,a_h_ftgoals_-14,a_h_ftgoals_-15,a_h_ftgoals_-16,a_h_ftgoals_-17,a_h_ftgoals_-18,a_h_shots_0,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_shots_-4,a_h_shots_-5,a_h_shots_-6,a_h_shots_-7,a_h_shots_-8,a_h_shots_-9,a_h_shots_-10,a_h_shots_-11,a_h_shots_-12,a_h_shots_-13,a_h_shots_-14,a_h_shots_-15,a_h_shots_-16,a_h_shots_-17,a_h_shots_-18,a_a_ftgoals_0,a_a_ftgoals_-1,a_a_ftgoals_-2,a_a_ftgoals_-3,a_a_ftgoals_-4,a_a_ftgoals_-5,a_a_ftgoals_-6,a_a_ftgoals_-7,a_a_ftgoals_-8,a_a_ftgoals_-9,a_a_ftgoals_-10,a_a_ftgoals_-11,a_a_ftgoals_-12,a_a_ftgoals_-13,a_a_ftgoals_-14,a_a_ftgoals_-15,a_a_ftgoals_-16,a_a_ftgoals_-17,a_a_ftgoals_-18,a_a_shots_0,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shots_-17,a_a_shots_-18,date,odds_hwin,odds_draw,odds_awin,result
375,wolverhampton-wanderers,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,14.0,14.0,10.0,9.0,12.0,11.0,8.0,6.0,9.0,11.0,12.0,12.0,8.0,8.0,8.0,11.0,12.0,24.0,19.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,3.0,0.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,2.0,11.0,12.0,9.0,22.0,15.0,7.0,10.0,6.0,20.0,12.0,10.0,17.0,13.0,11.0,12.0,12.0,3.0,5.0,16.0,sunderland,2.0,0.0,1.0,3.0,1.0,2.0,1.0,2.0,7.0,2.0,4.0,1.0,1.0,2.0,2.0,2.0,3.0,1.0,0.0,14.0,14.0,10.0,25.0,11.0,16.0,12.0,11.0,23.0,21.0,11.0,16.0,16.0,10.0,19.0,17.0,10.0,13.0,11.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,11.0,10.0,9.0,7.0,10.0,6.0,7.0,5.0,7.0,13.0,7.0,8.0,9.0,14.0,14.0,4.0,8.0,9.0,20.0,2010-05-09,2.30,3.25,3.08,hwin
376,west-ham-united,1.0,3.0,1.0,0.0,1.0,1.0,3.0,2.0,0.0,2.0,1.0,0.0,5.0,1.0,2.0,2.0,2.0,2.0,1.0,12.0,9.0,10.0,17.0,17.0,18.0,20.0,13.0,7.0,13.0,6.0,11.0,9.0,16.0,14.0,15.0,20.0,8.0,17.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,0.0,0.0,0.0,1.0,4.0,3.0,2.0,1.0,2.0,2.0,3.0,2.0,17.0,12.0,9.0,12.0,5.0,15.0,7.0,9.0,8.0,11.0,20.0,18.0,16.0,9.0,13.0,15.0,10.0,19.0,17.0,manchester-city,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,0.0,3.0,3.0,2.0,0.0,1.0,1.0,4.0,0.0,0.0,12.0,9.0,14.0,10.0,12.0,23.0,8.0,8.0,13.0,11.0,19.0,12.0,10.0,7.0,13.0,12.0,21.0,11.0,17.0,1.0,0.0,6.0,2.0,1.0,4.0,1.0,1.0,0.0,3.0,0.0,3.0,2.0,0.0,1.0,1.0,3.0,1.0,2.0,17.0,3.0,20.0,14.0,13.0,8.0,9.0,17.0,8.0,12.0,9.0,14.0,9.0,14.0,5.0,9.0,10.0,12.0,8.0,2010-05-09,4.10,3.51,1.85,draw
377,manchester-united,4.0,3.0,1.0,2.0,3.0,3.0,5.0,4.0,3.0,5.0,3.0,0.0,3.0,2.0,2.0,2.0,4.0,2.0,1.0,18.0,15.0,9.0,10.0,33.0,12.0,21.0,25.0,24.0,23.0,12.0,16.0,18.0,20.0,13.0,17.0,21.0,10.0,26.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,3.0,1.0,0.0,4.0,9.0,11.0,4.0,8.0,10.0,7.0,8.0,11

In [9]:
save_fp = PROCESSED_DATA_DIR / league / get_season[0] / str(league + '--' + get_season[0] + '.csv')
df.to_csv(save_fp, index=False)

In [10]:
df_load = pd.read_csv(save_fp, parse_dates = ['date'], index_col=None)
df_load.head()

,h,h_h_ftgoals_0,h_h_ftgoals_-1,h_h_ftgoals_-2,h_h_ftgoals_-3,h_h_ftgoals_-4,h_h_ftgoals_-5,h_h_ftgoals_-6,h_h_ftgoals_-7,h_h_ftgoals_-8,h_h_ftgoals_-9,h_h_ftgoals_-10,h_h_ftgoals_-11,h_h_ftgoals_-12,h_h_ftgoals_-13,h_h_ftgoals_-14,h_h_ftgoals_-15,h_h_ftgoals_-16,h_h_ftgoals_-17,h_h_ftgoals_-18,h_h_shots_0,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_a_ftgoals_0,h_a_ftgoals_-1,h_a_ftgoals_-2,h_a_ftgoals_-3,h_a_ftgoals_-4,h_a_ftgoals_-5,h_a_ftgoals_-6,h_a_ftgoals_-7,h_a_ftgoals_-8,h_a_ftgoals_-9,h_a_ftgoals_-10,h_a_ftgoals_-11,h_a_ftgoals_-12,h_a_ftgoals_-13,h_a_ftgoals_-14,h_a_ftgoals_-15,h_a_ftgoals_-16,h_a_ftgoals_-17,h_a_ftgoals_-18,h_a_shots_0,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_shots_-4,h_a_shots_-5,h_a_shots_-6,h_a_shots_-7,h_a_shots_-8,h_a_shots_-9,h_a_shots_-10,h_a_shots_-11,h_a_shots_-12,h_a_shots_-13,h_a_shots_-14,h_a_shots_-15,h_a_shots_-16,h_a_shots_-17,h_a_shots_-18,a,a_h_ftgoals_0,a_h_ftgoals_-1,a_h_ftgoals_-2,a_h_ftgoals_-3,a_h_ftgoals_-4,a_h_ftgoals_-5,a_h_ftgoals_-6,a_h_ftgoals_-7,a_h_ftgoals_-8,a_h_ftgoals_-9,a_h_ftgoals_-10,a_h_ftgoals_-11,a_h_ftgoals_-12,a_h_ftgoals_-13,a_h_ftgoals_-14,a_h_ftgoals_-15,a_h_ftgoals_-16,a_h_ftgoals_-17,a_h_ftgoals_-18,a_h_shots_0,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_shots_-4,a_h_shots_-5,a_h_shots_-6,a_h_shots_-7,a_h_shots_-8,a_h_shots_-9,a_h_shots_-10,a_h_shots_-11,a_h_shots_-12,a_h_shots_-13,a_h_shots_-14,a_h_shots_-15,a_h_shots_-16,a_h_shots_-17,a_h_shots_-18,a_a_ftgoals_0,a_a_ftgoals_-1,a_a_ftgoals_-2,a_a_ftgoals_-3,a_a_ftgoals_-4,a_a_ftgoals_-5,a_a_ftgoals_-6,a_a_ftgoals_-7,a_a_ftgoals_-8,a_a_ftgoals_-9,a_a_ftgoals_-10,a_a_ftgoals_-11,a_a_ftgoals_-12,a_a_ftgoals_-13,a_a_ftgoals_-14,a_a_ftgoals_-15,a_a_ftgoals_-16,a_a_ftgoals_-17,a_a_ftgoals_-18,a_a_shots_0,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shots_-17,a_a_shots_-18,date,odds_hwin,odds_draw,odds_awin,result
0,aston-villa,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wigan-athletic,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,1.66,3.51,5.33,awin
1,blackburn-rovers,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manchester-city,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,3.37,3.24,2.12,awin
2,bolton-wanderers,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sunderland,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1

In [11]:
df_load.tail()

,h,h_h_ftgoals_0,h_h_ftgoals_-1,h_h_ftgoals_-2,h_h_ftgoals_-3,h_h_ftgoals_-4,h_h_ftgoals_-5,h_h_ftgoals_-6,h_h_ftgoals_-7,h_h_ftgoals_-8,h_h_ftgoals_-9,h_h_ftgoals_-10,h_h_ftgoals_-11,h_h_ftgoals_-12,h_h_ftgoals_-13,h_h_ftgoals_-14,h_h_ftgoals_-15,h_h_ftgoals_-16,h_h_ftgoals_-17,h_h_ftgoals_-18,h_h_shots_0,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_a_ftgoals_0,h_a_ftgoals_-1,h_a_ftgoals_-2,h_a_ftgoals_-3,h_a_ftgoals_-4,h_a_ftgoals_-5,h_a_ftgoals_-6,h_a_ftgoals_-7,h_a_ftgoals_-8,h_a_ftgoals_-9,h_a_ftgoals_-10,h_a_ftgoals_-11,h_a_ftgoals_-12,h_a_ftgoals_-13,h_a_ftgoals_-14,h_a_ftgoals_-15,h_a_ftgoals_-16,h_a_ftgoals_-17,h_a_ftgoals_-18,h_a_shots_0,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_shots_-4,h_a_shots_-5,h_a_shots_-6,h_a_shots_-7,h_a_shots_-8,h_a_shots_-9,h_a_shots_-10,h_a_shots_-11,h_a_shots_-12,h_a_shots_-13,h_a_shots_-14,h_a_shots_-15,h_a_shots_-16,h_a_shots_-17,h_a_shots_-18,a,a_h_ftgoals_0,a_h_ftgoals_-1,a_h_ftgoals_-2,a_h_ftgoals_-3,a_h_ftgoals_-4,a_h_ftgoals_-5,a_h_ftgoals_-6,a_h_ftgoals_-7,a_h_ftgoals_-8,a_h_ftgoals_-9,a_h_ftgoals_-10,a_h_ftgoals_-11,a_h_ftgoals_-12,a_h_ftgoals_-13,a_h_ftgoals_-14,a_h_ftgoals_-15,a_h_ftgoals_-16,a_h_ftgoals_-17,a_h_ftgoals_-18,a_h_shots_0,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_shots_-4,a_h_shots_-5,a_h_shots_-6,a_h_shots_-7,a_h_shots_-8,a_h_shots_-9,a_h_shots_-10,a_h_shots_-11,a_h_shots_-12,a_h_shots_-13,a_h_shots_-14,a_h_shots_-15,a_h_shots_-16,a_h_shots_-17,a_h_shots_-18,a_a_ftgoals_0,a_a_ftgoals_-1,a_a_ftgoals_-2,a_a_ftgoals_-3,a_a_ftgoals_-4,a_a_ftgoals_-5,a_a_ftgoals_-6,a_a_ftgoals_-7,a_a_ftgoals_-8,a_a_ftgoals_-9,a_a_ftgoals_-10,a_a_ftgoals_-11,a_a_ftgoals_-12,a_a_ftgoals_-13,a_a_ftgoals_-14,a_a_ftgoals_-15,a_a_ftgoals_-16,a_a_ftgoals_-17,a_a_ftgoals_-18,a_a_shots_0,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shots_-17,a_a_shots_-18,date,odds_hwin,odds_draw,odds_awin,result
375,wolverhampton-wanderers,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,14.0,14.0,10.0,9.0,12.0,11.0,8.0,6.0,9.0,11.0,12.0,12.0,8.0,8.0,8.0,11.0,12.0,24.0,19.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,3.0,0.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,2.0,11.0,12.0,9.0,22.0,15.0,7.0,10.0,6.0,20.0,12.0,10.0,17.0,13.0,11.0,12.0,12.0,3.0,5.0,16.0,sunderland,2.0,0.0,1.0,3.0,1.0,2.0,1.0,2.0,7.0,2.0,4.0,1.0,1.0,2.0,2.0,2.0,3.0,1.0,0.0,14.0,14.0,10.0,25.0,11.0,16.0,12.0,11.0,23.0,21.0,11.0,16.0,16.0,10.0,19.0,17.0,10.0,13.0,11.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,11.0,10.0,9.0,7.0,10.0,6.0,7.0,5.0,7.0,13.0,7.0,8.0,9.0,14.0,14.0,4.0,8.0,9.0,20.0,2010-05-09,2.30,3.25,3.08,hwin
376,west-ham-united,1.0,3.0,1.0,0.0,1.0,1.0,3.0,2.0,0.0,2.0,1.0,0.0,5.0,1.0,2.0,2.0,2.0,2.0,1.0,12.0,9.0,10.0,17.0,17.0,18.0,20.0,13.0,7.0,13.0,6.0,11.0,9.0,16.0,14.0,15.0,20.0,8.0,17.0,1.0,2.0,0.0,1.0,3.0,2.0,0.0,0.0,0.0,0.0,1.0,4.0,3.0,2.0,1.0,2.0,2.0,3.0,2.0,17.0,12.0,9.0,12.0,5.0,15.0,7.0,9.0,8.0,11.0,20.0,18.0,16.0,9.0,13.0,15.0,10.0,19.0,17.0,manchester-city,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,0.0,3.0,3.0,2.0,0.0,1.0,1.0,4.0,0.0,0.0,12.0,9.0,14.0,10.0,12.0,23.0,8.0,8.0,13.0,11.0,19.0,12.0,10.0,7.0,13.0,12.0,21.0,11.0,17.0,1.0,0.0,6.0,2.0,1.0,4.0,1.0,1.0,0.0,3.0,0.0,3.0,2.0,0.0,1.0,1.0,3.0,1.0,2.0,17.0,3.0,20.0,14.0,13.0,8.0,9.0,17.0,8.0,12.0,9.0,14.0,9.0,14.0,5.0,9.0,10.0,12.0,8.0,2010-05-09,4.10,3.51,1.85,draw
377,manchester-united,4.0,3.0,1.0,2.0,3.0,3.0,5.0,4.0,3.0,5.0,3.0,0.0,3.0,2.0,2.0,2.0,4.0,2.0,1.0,18.0,15.0,9.0,10.0,33.0,12.0,21.0,25.0,24.0,23.0,12.0,16.0,18.0,20.0,13.0,17.0,21.0,10.0,26.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,3.0,1.0,0.0,4.0,9.0,11.0,4.0,8.0,10.0,7.0,8.0,11

In [15]:
df_load.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 159 columns):
h                  380 non-null object
h_h_ftgoals_0      380 non-null float64
h_h_ftgoals_-1     360 non-null float64
h_h_ftgoals_-2     340 non-null float64
h_h_ftgoals_-3     320 non-null float64
h_h_ftgoals_-4     300 non-null float64
h_h_ftgoals_-5     280 non-null float64
h_h_ftgoals_-6     260 non-null float64
h_h_ftgoals_-7     240 non-null float64
h_h_ftgoals_-8     220 non-null float64
h_h_ftgoals_-9     200 non-null float64
h_h_ftgoals_-10    180 non-null float64
h_h_ftgoals_-11    160 non-null float64
h_h_ftgoals_-12    140 non-null float64
h_h_ftgoals_-13    120 non-null float64
h_h_ftgoals_-14    100 non-null float64
h_h_ftgoals_-15    80 non-null float64
h_h_ftgoals_-16    60 non-null float64
h_h_ftgoals_-17    40 non-null float64
h_h_ftgoals_-18    20 non-null float64
h_h_shots_0        380 non-null float64
h_h_shots_-1       360 non-null float64
h_h_shots_-2       

In [13]:
df_load.describe()

,h_h_ftgoals_0,h_h_ftgoals_-1,h_h_ftgoals_-2,h_h_ftgoals_-3,h_h_ftgoals_-4,h_h_ftgoals_-5,h_h_ftgoals_-6,h_h_ftgoals_-7,h_h_ftgoals_-8,h_h_ftgoals_-9,h_h_ftgoals_-10,h_h_ftgoals_-11,h_h_ftgoals_-12,h_h_ftgoals_-13,h_h_ftgoals_-14,h_h_ftgoals_-15,h_h_ftgoals_-16,h_h_ftgoals_-17,h_h_ftgoals_-18,h_h_shots_0,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_a_ftgoals_0,h_a_ftgoals_-1,h_a_ftgoals_-2,h_a_ftgoals_-3,h_a_ftgoals_-4,h_a_ftgoals_-5,h_a_ftgoals_-6,h_a_ftgoals_-7,h_a_ftgoals_-8,h_a_ftgoals_-9,h_a_ftgoals_-10,h_a_ftgoals_-11,h_a_ftgoals_-12,h_a_ftgoals_-13,h_a_ftgoals_-14,h_a_ftgoals_-15,h_a_ftgoals_-16,h_a_ftgoals_-17,h_a_ftgoals_-18,h_a_shots_0,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_shots_-4,h_a_shots_-5,h_a_shots_-6,h_a_shots_-7,h_a_shots_-8,h_a_shots_-9,h_a_shots_-10,h_a_shots_-11,h_a_shots_-12,h_a_shots_-13,h_a_shots_-14,h_a_shots_-15,h_a_shots_-16,h_a_shots_-17,h_a_shots_-18,a_h_ftgoals_0,a_h_ftgoals_-1,a_h_ftgoals_-2,a_h_ftgoals_-3,a_h_ftgoals_-4,a_h_ftgoals_-5,a_h_ftgoals_-6,a_h_ftgoals_-7,a_h_ftgoals_-8,a_h_ftgoals_-9,a_h_ftgoals_-10,a_h_ftgoals_-11,a_h_ftgoals_-12,a_h_ftgoals_-13,a_h_ftgoals_-14,a_h_ftgoals_-15,a_h_ftgoals_-16,a_h_ftgoals_-17,a_h_ftgoals_-18,a_h_shots_0,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_shots_-4,a_h_shots_-5,a_h_shots_-6,a_h_shots_-7,a_h_shots_-8,a_h_shots_-9,a_h_shots_-10,a_h_shots_-11,a_h_shots_-12,a_h_shots_-13,a_h_shots_-14,a_h_shots_-15,a_h_shots_-16,a_h_shots_-17,a_h_shots_-18,a_a_ftgoals_0,a_a_ftgoals_-1,a_a_ftgoals_-2,a_a_ftgoals_-3,a_a_ftgoals_-4,a_a_ftgoals_-5,a_a_ftgoals_-6,a_a_ftgoals_-7,a_a_ftgoals_-8,a_a_ftgoals_-9,a_a_ftgoals_-10,a_a_ftgoals_-11,a_a_ftgoals_-12,a_a_ftgoals_-13,a_a_ftgoals_-14,a_a_ftgoals_-15,a_a_ftgoals_-16,a_a_ftgoals_-17,a_a_ftgoals_-18,a_a_shots_0,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shots_-17,a_a_shots_-18,odds_hwin,odds_draw,odds_awin
count,380.000000,360.000000,340.000000,320.000000,300.000000,280.000000,260.000000,240.00000,220.000000,200.000000,180.000000,160.000000,140.000000,120.000000,100.000000,80.000000,60.000000,40.000000,20.000000,380.000000,360.000000,340.000000,320.000000,300.000000,280.000000,260.000000,240.000000,220.000000,200.000000,180.000000,160.000000,140.000000,120.000000,100.000000,80.000000,60.000000,40.000000,20.000000,380.000000,360.000000,340.000000,320.000000,300.000000,280.000000,260.000000,240.000000,220.000000,200.000000,180.000000,160.000000,140.000000,120.000000,100.000000,80.000000,60.000000,40.000000,20.000000,380.000000,360.000000,340.000000,320.000000,300.000000,280.000000,260.000000,240.000000,220.000000,200.000000,180.000000,160.000000,140.000000,120.000000,100.000000,80.000000,60.000000,40.000000,20.000000,380.000000,360.000000,340.000000,320.000000,300.000000,280.000000,260.000000,240.000000,220.000000,200.000000,180.000000,160.000000,140.000000,120.000000,100.000000,80.000000,60.000000,40.000000,20.000000,380.000000,360.000000,340.000000,320.000000,300.000000,280.000000,260.000000,240.000000,220.000000,200.00000,180.000000,160.000000,140.000000,120.000000,100.000000,80.000000,60.000000,40.000000,20.000000,380.000000,360.000000,340.000000,320.000000,300.000000,280.000000,260.000000,240.000000,220.000000,200.000000,180.000000,160.000000,140.000000,120.000000,100.000000,80.000000,60.000000,40.000000,20.000000,380.000000,360.000000,340.000000,320.000000,300.000000,280.000000,260.000000,240.000000,220.000000,200.000000,180.000000,160.000000,140.000000,120.000000,100.00000,80.000000,60.000000,40.000000,20.000000,380.000000,380.000000,380.000000
mean,1.697368,1.691667,1.676471,1.728125,1.726667,1.692857,1.684615,1.71250,1.727273,1.755000,1.77222

In [33]:
df_load.iloc[44:50]

,h,h_h_ftgoals_0,h_h_ftgoals_-1,h_h_ftgoals_-2,h_h_ftgoals_-3,h_h_ftgoals_-4,h_h_ftgoals_-5,h_h_ftgoals_-6,h_h_ftgoals_-7,h_h_ftgoals_-8,h_h_ftgoals_-9,h_h_ftgoals_-10,h_h_ftgoals_-11,h_h_ftgoals_-12,h_h_ftgoals_-13,h_h_ftgoals_-14,h_h_ftgoals_-15,h_h_ftgoals_-16,h_h_ftgoals_-17,h_h_ftgoals_-18,h_h_shots_0,h_h_shots_-1,h_h_shots_-2,h_h_shots_-3,h_h_shots_-4,h_h_shots_-5,h_h_shots_-6,h_h_shots_-7,h_h_shots_-8,h_h_shots_-9,h_h_shots_-10,h_h_shots_-11,h_h_shots_-12,h_h_shots_-13,h_h_shots_-14,h_h_shots_-15,h_h_shots_-16,h_h_shots_-17,h_h_shots_-18,h_a_ftgoals_0,h_a_ftgoals_-1,h_a_ftgoals_-2,h_a_ftgoals_-3,h_a_ftgoals_-4,h_a_ftgoals_-5,h_a_ftgoals_-6,h_a_ftgoals_-7,h_a_ftgoals_-8,h_a_ftgoals_-9,h_a_ftgoals_-10,h_a_ftgoals_-11,h_a_ftgoals_-12,h_a_ftgoals_-13,h_a_ftgoals_-14,h_a_ftgoals_-15,h_a_ftgoals_-16,h_a_ftgoals_-17,h_a_ftgoals_-18,h_a_shots_0,h_a_shots_-1,h_a_shots_-2,h_a_shots_-3,h_a_shots_-4,h_a_shots_-5,h_a_shots_-6,h_a_shots_-7,h_a_shots_-8,h_a_shots_-9,h_a_shots_-10,h_a_shots_-11,h_a_shots_-12,h_a_shots_-13,h_a_shots_-14,h_a_shots_-15,h_a_shots_-16,h_a_shots_-17,h_a_shots_-18,a,a_h_ftgoals_0,a_h_ftgoals_-1,a_h_ftgoals_-2,a_h_ftgoals_-3,a_h_ftgoals_-4,a_h_ftgoals_-5,a_h_ftgoals_-6,a_h_ftgoals_-7,a_h_ftgoals_-8,a_h_ftgoals_-9,a_h_ftgoals_-10,a_h_ftgoals_-11,a_h_ftgoals_-12,a_h_ftgoals_-13,a_h_ftgoals_-14,a_h_ftgoals_-15,a_h_ftgoals_-16,a_h_ftgoals_-17,a_h_ftgoals_-18,a_h_shots_0,a_h_shots_-1,a_h_shots_-2,a_h_shots_-3,a_h_shots_-4,a_h_shots_-5,a_h_shots_-6,a_h_shots_-7,a_h_shots_-8,a_h_shots_-9,a_h_shots_-10,a_h_shots_-11,a_h_shots_-12,a_h_shots_-13,a_h_shots_-14,a_h_shots_-15,a_h_shots_-16,a_h_shots_-17,a_h_shots_-18,a_a_ftgoals_0,a_a_ftgoals_-1,a_a_ftgoals_-2,a_a_ftgoals_-3,a_a_ftgoals_-4,a_a_ftgoals_-5,a_a_ftgoals_-6,a_a_ftgoals_-7,a_a_ftgoals_-8,a_a_ftgoals_-9,a_a_ftgoals_-10,a_a_ftgoals_-11,a_a_ftgoals_-12,a_a_ftgoals_-13,a_a_ftgoals_-14,a_a_ftgoals_-15,a_a_ftgoals_-16,a_a_ftgoals_-17,a_a_ftgoals_-18,a_a_shots_0,a_a_shots_-1,a_a_shots_-2,a_a_shots_-3,a_a_shots_-4,a_a_shots_-5,a_a_shots_-6,a_a_shots_-7,a_a_shots_-8,a_a_shots_-9,a_a_shots_-10,a_a_shots_-11,a_a_shots_-12,a_a_shots_-13,a_a_shots_-14,a_a_shots_-15,a_a_shots_-16,a_a_shots_-17,a_a_shots_-18,date,odds_hwin,odds_draw,odds_awin,result
44,birmingham-city,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,8.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,13.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aston-villa,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-09-13,3.28,3.25,2.20,awin
45,fulham,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,everton,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-09-13,2.53,3.20,2.78,hwin
46,aston-villa,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,6.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,portsmouth,2.0,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,19.0,9.0,Na

## Functions to Cut Data to Defined Shape prior to PreProcessing

In [27]:
import re

def get_n_past_games(df_orig, past_games=2, dropna_rows=True):
    """
    Restrict historical records to the last n games
    If n = 2, then will get
    home team
        last 2 games played at home ie. h_h_feature-1, h_h_feature-2
        last 2 games played away ie h_a_feature-1, h_a_feature-2
    away team
        last 2 games played at home ie a_h_feature-1, a_h_feature-2
        last 2 games played away ie a_a_feature-1, a_a_feature-2
    where -1 is the most recent game prior to the current game, and -2 is the game before that
    the current games result is in result
    """
    df = df_orig.copy(deep=True)
    keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(hist_games+1) \
               or col.isalpha() \
               or 'odds' in col]
    
    # Rename the target columns to start with prefix 'res_'
    res_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
                and int(col.split('_')[-1]) == 0]
    res_cols.append('result')
    res_cols = [col.replace('0', 'res') for col in res_cols]
    print(res_cols)
    print(res_cols)
    df = df[keeper_cols]
    return df
    
df_cut = get_n_past_games(df_load)
# # Restrict Historical records to last 4 games per team
# df_cut = df_load.copy(deep=True)
# hist_games = 2
# keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
#                and int(col.split('_')[-1]) > -(hist_games+1) \
#                and int(col.split('_')[-1]) < 0 \
#                or col.isalpha() \
#                or 'odds' in col]
# df_cut = df_cut[keeper_cols]
df_cut.head()

['h_h_ftgoals_res', 'h_h_shots_res', 'h_a_ftgoals_res', 'h_a_shots_res', 'a_h_ftgoals_res', 'a_h_shots_res', 'a_a_ftgoals_res', 'a_a_shots_res', 'result']
['h_h_ftgoals_res', 'h_h_shots_res', 'h_a_ftgoals_res', 'h_a_shots_res', 'a_h_ftgoals_res', 'a_h_shots_res', 'a_a_ftgoals_res', 'a_a_shots_res', 'result']


,h,h_h_ftgoals_0,h_h_ftgoals_-1,h_h_ftgoals_-2,h_h_shots_0,h_h_shots_-1,h_h_shots_-2,h_a_ftgoals_0,h_a_ftgoals_-1,h_a_ftgoals_-2,h_a_shots_0,h_a_shots_-1,h_a_shots_-2,a,a_h_ftgoals_0,a_h_ftgoals_-1,a_h_ftgoals_-2,a_h_shots_0,a_h_shots_-1,a_h_shots_-2,a_a_ftgoals_0,a_a_ftgoals_-1,a_a_ftgoals_-2,a_a_shots_0,a_a_shots_-1,a_a_shots_-2,date,odds_hwin,odds_draw,odds_awin,result
0,aston-villa,0.0,NaN,NaN,11.0,NaN,NaN,2.0,NaN,NaN,14.0,NaN,NaN,wigan-athletic,0.0,NaN,NaN,11.0,NaN,NaN,2.0,NaN,NaN,14.0,NaN,NaN,2009-08-15,1.66,3.51,5.33,awin
1,blackburn-rovers,0.0,NaN,NaN,17.0,NaN,NaN,2.0,NaN,NaN,8.0,NaN,NaN,manchester-city,0.0,NaN,NaN,17.0,NaN,NaN,2.0,NaN,NaN,8.0,NaN,NaN,2009-08-15,3.37,3.24,2.12,awin
2,bolton-wanderers,0.0,NaN,NaN,11.0,NaN,NaN,1.0,NaN,NaN,20.0,NaN,NaN,sunderland,0.0,NaN,NaN,11.0,NaN,NaN,1.0,NaN,NaN,20.0,NaN,NaN,2009-08-15,2.24,3.20,3.15,awin
3,chelsea,2.0,NaN,NaN,26.0,NaN,NaN,1.0,NaN,NaN,7.0,NaN,NaN,hull-city,2.0,NaN,NaN,26.0,NaN,NaN,1.0,NaN,NaN,7.0,NaN,NaN,2009-08-15,1.17,6.26,16.39,hwin
4,everton,1.0,NaN,NaN,8.0,NaN,NaN,6.0,NaN,NaN,15.0,NaN,NaN,arsenal,1.0,NaN,NaN,8.0,NaN,NaN,6.0,NaN,NaN,15.0,NaN,NaN,2009-08-15,3.07,3.21,2.28,awin


In [19]:
import re

def get_n_past_games(df_orig, past_games=2, dropna_rows=True):
    """
    Restrict historical records to the last n games
    If n = 2, then will get
    home team
        last 2 games played at home ie. h_h_feature-1, h_h_feature-2
        last 2 games played away ie h_a_feature-1, h_a_feature-2
    away team
        last 2 games played at home ie a_h_feature-1, a_h_feature-2
        last 2 games played away ie a_a_feature-1, a_a_feature-2
    where -1 is the most recent game prior to the current game, and -2 is the game before that
    the current games result is in result
    """
    df = df_orig.copy(deep=True)
    keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(hist_games+1) \
               and int(col.split('_')[-1]) < 0 \
               or col.isalpha() \
               or 'odds' in col]
df_cut = df_cut[keeper_cols]
    

# Restrict Historical records to last 4 games per team
df_cut = df_load.copy(deep=True)
hist_games = 2
keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(hist_games+1) \
               and int(col.split('_')[-1]) < 0 \
               or col.isalpha() \
               or 'odds' in col]
df_cut = df_cut[keeper_cols]
df_cut.head(10)

,h,h_h_ftgoals_-1,h_h_ftgoals_-2,h_h_shots_-1,h_h_shots_-2,h_a_ftgoals_-1,h_a_ftgoals_-2,h_a_shots_-1,h_a_shots_-2,a,a_h_ftgoals_-1,a_h_ftgoals_-2,a_h_shots_-1,a_h_shots_-2,a_a_ftgoals_-1,a_a_ftgoals_-2,a_a_shots_-1,a_a_shots_-2,date,odds_hwin,odds_draw,odds_awin,result
0,aston-villa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wigan-athletic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,1.66,3.51,5.33,awin
1,blackburn-rovers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manchester-city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,3.37,3.24,2.12,awin
2,bolton-wanderers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sunderland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,2.24,3.20,3.15,awin
3,chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hull-city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,1.17,6.26,16.39,hwin
4,everton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,3.07,3.21,2.28,awin
5,portsmouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fulham,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,2.53,3.18,2.73,awin
6,stoke-city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,burnley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,1.91,3.29,4.02,hwin
7,wolverhampton-wanderers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,west-ham-united,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-15,2.57,3.19,2.68,awin
8,manchester-united,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,birmingham-city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-16,1.19,5.91,14.99,hwin
9,tottenham-hotspur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,liverpool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-16,3.13,3.22,2.23,hwin
